In [11]:
import numpy as np
import math 

In [12]:
# set random seed for reproducibility 
np.random.seed(42)

In [13]:
from activation_function import ActivationFunction
from loss_function import LossFunction

In [14]:
class Neuron:
    """
    A simple artificial neuron that computes a weighted sum of its inputs, applies an activation function, and produces an output.
    """

    def __init__(self, num_inputs, activation ='relu'):
        """
        Initialize the neuron with random weights and bias.
        """

        # xavier initialization for weights
        limit = 1/math.sqrt(num_inputs)
        self.weights = np.random.uniform(-limit,limit,num_inputs)

        # Bias initialization
        self.bias = np.random.uniform(-limit, limit)

        self.inputs = None
        self.output = None

        self.activation = ActivationFunction.get_activation(activation)
        self.activation_derivative = ActivationFunction.get_activation_derivative(activation)


    def forward(self, inputs):
        """
        Compute output of the neurons in the layer given the inputs.
        """

        # preserve copy of original inputs 
        self.inputs = np.array(inputs)

        # compute weighted sum
        weighted_sum = np.dot(self.weights, self.inputs) + self.bias

        # applying ReLU by default
        self.output = self.activation(weighted_sum)

        return self.output


    def __str__(self):
        """String representation of the neuron"""
        return f"Neuron(weights={[round(w, 3) for w in self.weights]}, bias={round(self.bias, 3)})"



In [15]:
class Layer:
    """
    A layer of neurons in a neural network.
    """

    def __init__(self, num_neurons, num_inputs_per_neuron=None, activation='relu', is_output=False):
        """
        Initialize the layer with given number of neurons, each with specified number of inputs.

        Args: 
            num_neurons (int): Number of neurons in the layer.
            num_inputs_per_neuron (int): Number of inputs each neuron receives.
            is_output (bool): Flag indicating if this layer is the output layer.
        """

        self.num_neurons = num_neurons
        self.num_inputs_per_neuron = num_inputs_per_neuron
        self.is_output = is_output

        # creating neurons for the layer
        self.neurons = [Neuron(num_inputs_per_neuron, activation) for _ in range(num_neurons)]
        self.inputs = None
        self.outputs = None

        
    def forward(self, inputs):
        """
        Forwards pass through layers sequentially by computing outputs of all neurons in the layer.
        """

        self.inputs = np.array(inputs)

        # get output from each neuron
        self.outputs = np.array([neuron.forward(inputs) for neuron in self.neurons])

        return self.outputs



    def __str__(self):
        """String representation of the layer"""
        layer_type = "Output" if self.is_output else "Hidden"
        return f"{layer_type} Layer ({self.num_neurons} neurons, {self.num_inputs_per_neuron} inputs each)"
        

In [ ]:
class Network:
    """
        Neural Network consisting of multiple layers.
        Basic idea of Multi-Layer perceptron
    """

    def __init__(self, loss_name='mse'):


        self. layers = []
        self.loss_fn = LossFunction.get_loss_function(loss_name)

    def add_layer(self, num_neurons, num_inputs=None, activation = 'relu',is_output=False):
        """
        Initialize layers and add to the network.

        Args:
            num_neurons (int): Number of neurons in the layer.
            num_inputs (int): Number of inputs each neuron receives. Required for the first layer.
            activation (str): Activation function to be used in the layer.
            is_output (bool): Flag indicating if this layer is the output layer.
        """

        if not self.layers and num_inputs is None:
            raise ValueError("Number of inputs must be specified for the first layer.")
        
        num_inputs_per_neuron = num_inputs if not self.layers else self.layers[-1].num_neurons  # get from previous layer
        layer = Layer(num_neurons, num_inputs_per_neuron, activation, is_output)
        self.layers.append(layer)

    def forward(self, inputs):
        """
        Forward pass through the entire network.
        """

        current_input = np.array(inputs)
        for layer in self.layers: 
            current_input = layer.forward(current_input)
        return current_input

    def predict(self, X):
        """Predict outputs for given inputs X."""
        X = np.array(X)
        return np.array([self.forward(x) for x in X])
        
    
    # fit method for training 
    def fit(self, X,y, epochs =100, learning_rate =0.01):
        """
        Train the network using Gradient descent
        """
        for epoch in range(epochs):
            total_loss = 0
            for inputs, targets in zip(X,y):
                
                # forward pass
                predictions = self.predict(inputs)

                # compute loss 
                loss = self.loss_fn(predictions, targets)
                total_loss += loss

                # backward pass 
                # self.backward(predictions, targets)

            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {total_loss/len(X)}")

        
        # final outcome 
        print(f"Final Loss: {total_loss/len(X)}")

    

    def __str__(self):
        """String representation of the network"""
        return f"Network(layers={self.num_layers}, layer_sizes={self.layer_sizes})"


In [18]:
nw = Network(loss_name='huber')

nw.add_layer(num_neurons=2, num_inputs=2, activation='relu') 
nw.add_layer(num_neurons=3, activation='tanh')
nw.add_layer(num_neurons=1, activation='linear', is_output=True)


print("Network Architecture:")
for i, layer in enumerate(nw.layers):
    print(f"Layer {i+1}: {layer}")
    for j, neuron in enumerate(layer.neurons):
        print(f"  Neuron {j+1}: {neuron}")


Network Architecture:
Layer 1: Hidden Layer (2 neurons, 2 inputs each)
  Neuron 1: Neuron(weights=[-0.177, 0.637], bias=0.328)
  Neuron 2: Neuron(weights=[0.14, -0.486], bias=-0.486)
Layer 2: Hidden Layer (3 neurons, 2 inputs each)
  Neuron 1: Neuron(weights=[-0.625, 0.518], bias=0.143)
  Neuron 2: Neuron(weights=[0.294, -0.678], bias=0.665)
  Neuron 3: Neuron(weights=[0.47, -0.407], bias=-0.45)
Layer 3: Output Layer (1 neurons, 3 inputs each)
  Neuron 1: Neuron(weights=[-0.366, -0.226, 0.029], bias=-0.079)


In [19]:
input_data = np.array([0.5, -1.5])
output = nw.predict(input_data)
print(f"Network output: {output}")

# You can also test layer by layer:
print("\nLayer by layer:")
layer1_output = nw.layers[0].forward(input_data)
print(f"Layer 1 output: {layer1_output}")

layer2_output = nw.layers[1].forward(layer1_output)
print(f"Layer 2 (final) output: {layer2_output}")

Network output: [[[-0.22699173 -0.1461266 ]]

 [[-0.15629815 -0.29327881]]]

Layer by layer:
Layer 1 output: [0.         0.31295952]
Layer 2 (final) output: [ 0.29594065  0.42384324 -0.52068839]


In [20]:
X_train = [[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]]
y_train = [[0.3], [0.7], [1.1]]

nw.fit(X_train, y_train, epochs=50, learning_rate=0.01)

Epoch 0, Loss: 0.4366548085687097
Epoch 10, Loss: 0.4366548085687097
Epoch 20, Loss: 0.4366548085687097
Epoch 30, Loss: 0.4366548085687097
Epoch 40, Loss: 0.4366548085687097
Final Loss: 0.4366548085687097


In [21]:
nw.predict([[0.2, 0.4],[0.5,0.6]])

array([[-0.16543196],
       [-0.15094246]])